In [91]:
from langchain_openai import ChatOpenAI
import pandas as pd
import os
from typing import List
from enum import Enum

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.enum import EnumOutputParser


with open('config/config.txt') as f:
    for line in f:
        env_data = line.split(',')
        os.environ[env_data[0]] = env_data[1]

llm = ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0.7)

reviews = pd.read_csv('data/review_sample.csv', index_col=0)

In [96]:
def extract_significant_ideas(text):
    # Define the prompt to instruct the model to extract significant ideas
    prompt = f"Please summarize the following text by extracting the significant ideas as short sentences. SEPARATE EACH SENTENCE WITH COMMAS ONLY:\n\n{text}"
    
    # Use LangChain's llm to call the OpenAI API
    response = llm.invoke(prompt)

    return response

ideas = {}

# Extract significant ideas
for i in range(5):
    significant_ideas = extract_significant_ideas(reviews.text[i])
    ideas[reviews.review_id[i]] = significant_ideas.content.split(',')

In [97]:
ideas

{'QEkyAdsrVo7tEsANDN7O9g': ['Missed Marvin',
  ' enjoyed chocolate shake',
  ' went later in the day',
  ' best time to go is when they open',
  ' Marvin is entertaining.'],
 'kIViMGM4JTHHozAE1r9boA': ['Best breakfast in a long time',
  ' disappointed with breakfast before',
  ' Cammellias was amazing',
  ' had the omelet at the bottom of the menu',
  ' so good! Brother had the Mexican',
  ' very tasty',
  ' pecan omelet is to die for',
  ' thinner than most',
  ' order ASAP',
  ' busy',
  ' service is personable and awesome.'],
 'LpHt30LaQHAtBs-j_EYUUw': ['Recently visited new French Quarter location',
  ' enjoyed as much as Carrollton location',
  ' Get Manhattan Omelette',
  ' To die for.'],
 'WqCjrz_fV-n6y_nDxme2lA': ['One of my favorite places',
  ' Marvin is high energy and fun',
  ' awesome service',
  ' food is amazing',
  ' best breakfast place',
  ' New Orleans landmark',
  ' well worth the wait',
  ' huge smiley face.'],
 'J8kc5pwyAtpl1Wm91RopYQ': ['Total disappointment',
  

In [100]:
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field



In [119]:
class Boolean(Enum):
    TRUE = "TRUE"
    FALSE = "FALSE"

def merge_related_ideas(idea_0, idea_1):
    parser = EnumOutputParser(enum=Boolean)
    prompt = PromptTemplate.from_template(
        """
        Please look at the following two ideas and decide whether they express the same or a very similar thing: 
        
        Idea 1: {idea_0}
        
        Idea 2: {idea_1}

        Instructions: {instructions}
        """
    ).partial(instructions=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"idea_0": idea_0, "idea_1": idea_1})
    return response.value == 'TRUE'

a = ideas['LpHt30LaQHAtBs-j_EYUUw'][-1]
b = ideas['kIViMGM4JTHHozAE1r9boA'][4]

r = merge_related_ideas(a, b)

In [120]:
r

False

In [121]:
a, b

(' To die for.', ' so good! Brother had the Mexican')